In [ ]:
%pylab inline
%pip install ucimlrepo

In [ ]:
# ライブラリの読み込み
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
from torchvision import transforms
from sklearn.model_selection import train_test_split
from ucimlrepo import fetch_ucirepo


In [ ]:
# 必要なデータセットの読み込み
# データの取得
X, y = torch.load('crown.pt')
train_data, test_data, train_labels, test_labels = train_test_split(X, y, test_size=0.2, random_state=42)

# データの前処理
class OpticalDigitsDataset(data.Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        sample = self.X[idx].reshape(8, 8).astype('float32')
        label = self.y[idx]

        if self.transform:
            sample = self.transform(sample)

        return sample, label

# テンソルへの変換と標準化
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# データのロード
train_dataset = OpticalDigitsDataset(train_data, train_labels, transform=transform)
test_dataset = OpticalDigitsDataset(test_data, test_labels, transform=transform)
train_loader = data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=1)
test_loader = data.DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=1)

# モデルの定義
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2) #pooling:特徴量を減らす
        self.fc1 = nn.Linear(32 * 2 * 2, 120)
        self.fc2 = nn.Linear(120, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 32 * 2 * 2)
        x = self.relu(self.fc1(x))
        x = self.fc2(x).unsqueeze(-1)
        return x

# モデルの初期化
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# トレーニングループ
def train(model, train_loader, criterion, optimizer, epoch):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}], Loss: {running_loss/len(train_loader):.4f}')

# 評価
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total:.2f}%')

# 学習の実行
num_epochs = 10
for epoch in range(num_epochs):
    train(model, train_loader, criterion, optimizer, epoch)
    evaluate(model, test_loader)
